# wps_bccaq
A process that runs the [bccaq.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/BCCAQ.R) function from ClimDown.

Bias Correction/Constructed Analogues with Quantile mapping reordering (BCCAQ) is composed of the following steps.
- Constructed Analogues (CA)
- Climate Imprint (CI)
- Quantile Delta Mapping (QDM)
- Rerank


In [1]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from tempfile import NamedTemporaryFile
from datetime import date
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [2]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

Using chickadee on https://docker-dev03.pcic.uvic.ca/twitcher/ows/proxy/chickadee/wps


In [3]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [4]:
# NBVAL_IGNORE_OUTPUT
chickadee.bccaq?

Signature:
chickadee.bccaq(
    gcm_file,
    obs_file=None,
    varname=None,
    num_cores='4',
    loglevel='INFO',
    units_bool=True,
    n_pr_bool=True,
    tasmax_units='celsius',
    tasmin_units='celsius',
    pr_units='kg m-2 d-1',
    max_gb=1.0,
    start_date=datetime.date(1971, 1, 1),
    end_date=datetime.date(2005, 12, 31),
    num_analogues='30',
    delta_days='45',
    trimmed_mean=0.0,
    tol=0.1,
    multiyear=True,
    expand_multiyear=True,
    multiyear_window_length='30',
    trace=0.005,
    jitter_factor=0.01,
    pr_tau='1001',
    tasmax_tau='101',
    tasmin_tau='101',
    pr_seasonal=True,
    tasmax_seasonal=False,
    tasmin_seasonal=False,
    pr_ratio=True,
    tasmax_ratio=False,
    tasmin_ratio=False,
    out_file=None,
    output_formats=None,
)
Docstring:
Full statistical downscaling of coarse scale global climate model (GCM) output to a fine spatial resolution

Parameters
----------
gcm_file : ComplexData:mimetype:`application/x-netcdf`, :mime

### Run the bccaq process

In [5]:
with NamedTemporaryFile(
    suffix=".nc", prefix="output_", dir="/tmp", delete=True
) as out_file:
    output = chickadee.bccaq(
        gcm_file=resource_filename("tests", "/data/tiny_gcm.nc"), 
        obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
        varname="tasmax", 
        end_date=date(1972, 12, 31),
        out_file=out_file.name,
        num_cores=4
    )

In [7]:
# Use asobj=True to access the output file contents as a dataset
output_dataset = output.get(asobj=True)[0]

### Test for expected output

In [8]:
expected_data = Dataset(resource_filename("tests","/data/bccaq_expected_output.nc"))
for key, value in expected_data.dimensions.items():
    assert str(output_dataset.dimensions[key]) == str(value)